In [69]:
from tqdm.auto import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets

from typing import Optional
import os
import csv
import nest_asyncio
nest_asyncio.apply()

In [70]:
from datasets import load_dataset

eval_dataset = load_dataset("csv", data_files="data/eval.csv")["train"]

In [71]:
print(eval_dataset)

Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'groundedness_score', 'groundedness_eval', 'relevance_score', 'relevance_eval', 'standalone_score', 'standalone_eval'],
    num_rows: 52
})


In [72]:
OUTPUT_FILE = "data/eval_rag_results.csv"

def load_existing_results(file_path):
    dataset = []
    existing_questions = set()
    if os.path.exists(file_path):
        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_questions.add(row["user_input"])
                # parse retrieved_contexts lại từ JSON
                row["retrieved_contexts"] = json.loads(row["retrieved_contexts"])
                dataset.append(row)
    return existing_questions, dataset


def append_result_to_csv(file_path, row, fieldnames):
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    verbose: Optional[bool] = False
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    fieldnames = ["user_input", "retrieved_contexts", "response", "reference"]

    processed_questions, dataset = load_existing_results(OUTPUT_FILE)
    print(f"Đã có {len(processed_questions)} câu hỏi xử lý trước đó, sẽ bỏ qua chúng.")


    for i, example in enumerate(tqdm(eval_dataset)):
        question = example["question"]
        print(question)
        if question in processed_questions:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4()),
            "chat_history": []
        }

        # try:
        relevant_docs, answer = asyncio.run(call_chat_once(payload))
        # except Exception as e:
        #     print(f"Error from question: '{question}': {e}")

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')

        row = {
            "user_input": question,
            "retrieved_contexts": json.dumps(
                [doc.get('page_content') for doc in relevant_docs],
                ensure_ascii=False
            ),
            "response": answer,
            "reference": example['answer']
        }
        append_result_to_csv(OUTPUT_FILE, row, fieldnames)
        dataset.append({
            "user_input": question,
            "retrieved_contexts": json.loads(row["retrieved_contexts"]),
            "response": answer,
            "reference": example['answer']
        })
    return dataset

In [ ]:
data = run_rag_tests(eval_dataset=eval_dataset)


Đã có 5 câu hỏi xử lý trước đó, sẽ bỏ qua chúng.


  0%|          | 0/52 [00:00<?, ?it/s]

Who proposed the hybrid Vision Transformer model?
Who are the authors of the Longformer paper?
How many layers does the distilbert-base-uncased model have?
What is the style of TPU access when using Google Colab?
What is the default label inferred by Gradio for the input parameter in the GUI?
What does the 'beta_start' parameter value mean in DDPMScheduler?


2025-08-13 11:46:30,077 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 12%|█▏        | 6/52 [00:08<01:01,  1.33s/it]

How do you register a custom Resnet model to the auto classes in Transformers?


...

What is the Top 1 Accuracy of tv_resnet152 on ImageNet?


2025-08-13 11:47:48,670 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 50%|█████     | 26/52 [01:26<01:26,  3.32s/it]

Under which license is the HuggingFace Transformers library released?


2025-08-13 11:47:51,302 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 52%|█████▏    | 27/52 [01:29<01:17,  3.11s/it]

What is the model structure of GPTSAN?


In [ ]:
from ragas import EvaluationDataset
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    FactualCorrectness
)
evaluation_dataset = EvaluationDataset.from_list(data)

# Khởi tạo các metric
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]
# metrics = [FactualCorrectness()]

In [ ]:
print(len(evaluation_dataset))

5


In [ ]:
from app.config import configs

import os
os.environ['OPENAI_API_KEY'] = configs.OPENAI_API_KEY

result = evaluate(
    evaluation_dataset,
    metrics=metrics,
)

print(result) 

Evaluating:   0%|          | 0/20 [00:00<?, ?it/s]2025-08-13 11:44:14,131 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,278 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,389 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,407 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,437 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,453 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,462 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,477 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 11:44:14,503 - INFO - HTTP Request:

{'faithfulness': 0.8000, 'answer_relevancy': 0.9630, 'context_precision': 0.7500, 'context_recall': 0.8000}
